In [15]:
import pandas as pd

In [17]:
df = pd.read_csv('online_retail_II.csv')

In [18]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


In [20]:
# 기술통계
df.describe()

,Quantity,Price,Customer ID
count,1.067371e+06,1.067371e+06,824364.000000
mean,9.938898e+00,4.649388e+00,15324.638504
std,1.727058e+02,1.235531e+02,1697.464450
min,-8.099500e+04,-5.359436e+04,12346.000000
25%,1.000000e+00,1.250000e+00,13975.000000
50%,3.000000e+00,2.100000e+00,15255.000000
75%,1.000000e+01,4.150000e+00,16797.000000
max,8.099500e+04,3.897000e+04,18287.000000


In [21]:
# 결측치
df.isnull().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

In [22]:
# 결측치 비율
df.isnull().mean() * 100

Invoice         0.000000
StockCode       0.000000
Description     0.410541
Quantity        0.000000
InvoiceDate     0.000000
Price           0.000000
Customer ID    22.766873
Country         0.000000
dtype: float64

In [27]:
# InvoiceDate 타입 datetime 으로 변경
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
 8   TotalPrice   1067371 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 73.3+ MB


In [28]:
# 전체 주문금액 변수 할당
df['TotalPrice'] = df['Quantity'] * df['Price']
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


In [37]:
# 일별 매출
daily_sales = df.groupby(df['InvoiceDate'].dt.date)['TotalPrice'].sum().reset_index()
daily_sales.head()

,InvoiceDate,TotalPrice
0,2009-12-01,53173.03
1,2009-12-02,62763.59
2,2009-12-03,68093.05
3,2009-12-04,40346.40
4,2009-12-05,9803.05


In [39]:
# 월별 매출
monthly_sales = df.groupby(df['InvoiceDate'].dt.to_period('M'))['TotalPrice'].sum().reset_index()
monthly_sales.head()

,InvoiceDate,TotalPrice
0,2009-12,799847.110
1,2010-01,624032.892
2,2010-02,533091.426
3,2010-03,765848.761
4,2010-04,590580.432


In [45]:
# 국가별 매출
country_total_sales = df.groupby('Country')['TotalPrice'].sum().reset_index()
country_total_sales.head()

,Country,TotalPrice
0,Australia,167129.07
1,Austria,23177.60
2,Bahrain,2861.55
3,Belgium,63574.49
4,Bermuda,1253.14


In [49]:
# MAU

# 1. 결측치 처리
df = df.dropna(subset=['Customer ID'])

# 2. 월별 고유 고객 수 계산
mau = df.groupby(df['InvoiceDate'].dt.to_period('M'))['Customer ID'].nunique().reset_index()

# 3. 결과 확인
mau

,InvoiceDate,Customer ID
0,2009-12,1045
1,2010-01,786
2,2010-02,807
3,2010-03,1111
4,2010-04,998
5,2010-05,1062
6,2010-06,1095
7,2010-07,988
8,2010-08,964
9,2010-09,1202


In [52]:
# 1. 결측치 처리
df = df.dropna(subset=['Customer ID'])

# 2. 각 고객의 구매 횟수 계산
customer_purchase_counts = df.groupby('Customer ID')['InvoiceDate'].nunique().reset_index()

# 3. 반복 구매 고객 추출 (2회 이상 구매한 고객)
repeat_customers = customer_purchase_counts[customer_purchase_counts['InvoiceDate'] > 1]

# 4. 반복 고객 수
print("반복 구매 고객 수:")
print(repeat_customers)

# 5. 총 반복 구매 고객 수
total_repeat_customers = len(repeat_customers)
print(f"총 반복 구매 고객 수: {total_repeat_customers}")

반복 구매 고객 수:
      Customer ID  InvoiceDate
0         12346.0           17
1         12347.0            8
2         12348.0            5
3         12349.0            5
6         12352.0           13
...           ...          ...
5936      18282.0            3
5937      18283.0           22
5938      18284.0            2
5940      18286.0            3
5941      18287.0            8

[4482 rows x 2 columns]
총 반복 구매 고객 수: 4482


In [53]:
# 결측치 처리: CustomerID가 NaN인 행 제거
df = df.dropna(subset=['Customer ID'])

# 각 고객의 마지막 구매 날짜 계산
last_purchase = df.groupby('Customer ID')['InvoiceDate'].max().reset_index()

# 현재 날짜 설정 (예: 2024-01-01)
current_date = pd.to_datetime('2024-09-24')

# 이탈 고객 조건 설정: 마지막 구매일로부터 6개월 이상 경과한 고객
last_purchase['Churned'] = last_purchase['InvoiceDate'] < (current_date - pd.DateOffset(months=6))

# 이탈 고객 필터링
churned_customers = last_purchase[last_purchase['Churned'] == True]

# 결과 확인
print("이탈 고객 수:")
print(churned_customers)

# 이탈 고객 수 출력
total_churned_customers = len(churned_customers)
print(f"총 이탈 고객 수: {total_churned_customers}")

이탈 고객 수:
      Customer ID         InvoiceDate  Churned
0         12346.0 2011-01-18 10:17:00     True
1         12347.0 2011-12-07 15:52:00     True
2         12348.0 2011-09-25 13:13:00     True
3         12349.0 2011-11-21 09:51:00     True
4         12350.0 2011-02-02 16:01:00     True
...           ...                 ...      ...
5937      18283.0 2011-12-06 12:02:00     True
5938      18284.0 2010-10-06 12:31:00     True
5939      18285.0 2010-02-17 10:24:00     True
5940      18286.0 2010-08-20 11:57:00     True
5941      18287.0 2011-10-28 09:29:00     True

[5942 rows x 3 columns]
총 이탈 고객 수: 5942


In [54]:
# 결측치 처리: CustomerID가 NaN인 행 제거
df = df.dropna(subset=['Customer ID'])

# 첫 구매 날짜 계산
df['FirstPurchaseDate'] = df.groupby('Customer ID')['InvoiceDate'].transform('min')

# 첫 구매 연도 및 월 추출
df['CohortMonth'] = df['FirstPurchaseDate'].dt.to_period('M')

# 구매 연도 및 월 추출
df['PurchaseMonth'] = df['InvoiceDate'].dt.to_period('M')

# 코호트 테이블 생성
cohort_data = df.groupby(['CohortMonth', 'PurchaseMonth'])['Customer ID'].nunique().reset_index()

# 코호트 크기 계산
cohort_size = cohort_data.groupby('CohortMonth')['Customer ID'].first().reset_index()
cohort_size.columns = ['CohortMonth', 'CohortSize']

# 코호트 테이블에 코호트 크기 추가
cohort_data = cohort_data.merge(cohort_size, on='CohortMonth')

# 유지율 계산
cohort_data['Retention'] = cohort_data['Customer ID'] / cohort_data['CohortSize']

# 결과 확인
print("코호트 분석 결과:")
print(cohort_data.pivot(index='CohortMonth', columns='PurchaseMonth', values='Retention'))

코호트 분석 결과:
PurchaseMonth  2009-12  2010-01   2010-02   2010-03   2010-04   2010-05  \
CohortMonth                                                               
2009-12            1.0  0.37512  0.342584  0.427751  0.392344  0.390431   
2010-01            NaN  1.00000  0.218274  0.302030  0.304569  0.279188   
2010-02            NaN      NaN  1.000000  0.300275  0.225895  0.303030   
2010-03            NaN      NaN       NaN  1.000000  0.217890  0.259174   
2010-04            NaN      NaN       NaN       NaN  1.000000  0.230241   
2010-05            NaN      NaN       NaN       NaN       NaN  1.000000   
2010-06            NaN      NaN       NaN       NaN       NaN       NaN   
2010-07            NaN      NaN       NaN       NaN       NaN       NaN   
2010-08            NaN      NaN       NaN       NaN       NaN       NaN   
2010-09            NaN      NaN       NaN       NaN       NaN       NaN   
2010-10            NaN      NaN       NaN       NaN       NaN       NaN   
2010-11       